In [144]:
import numpy as np
from numpy.random import normal

# find M closest

def closest(x0,x1,sigma=1.0,M=5):
    # which ensembles x0 are closzest to x1
    # measure distance
    d=np.abs(x0-x1)
    
    # just see how many are in spec
    w = np.where(d<3*sigma)
    if (len(w[0])<M):
        return None
    print(f'{len(w[0])} samples within spec.')
    
    # sort the best M
    indices=np.argsort(d[:M])
    # M lowest distance
    return d[indices]

def gaussian(x,mu,sigma):
    z=(x-mu)/(2.*sigma)
    g = np.exp(-z*z)
    return g/g.sum()

def varsolution(x0,x1,s0=1.0,s1=1.0):
    num = (x0/(s0*s0)) + (x1/(s1*s1))
    den = (1.0/(s0*s0)) + (1.0/(s1*s1))
    return num/den,np.sqrt(1./den)

# Issues with matching N samples from a distribution as a DA constraint

## Example 1

We have a 1D Gaussian prior $B$ with mean $x_b=0$ and standard deviation $σ_b=1.

We take an observation of $x$, $x_{obs}=5$ with measurement uncertainty $σ_{obs}=1$. 

We want to use DA to estimate the posterior distribution, based on the prior constraint and the observation. We have a *truth* here, in that the variational approach for Gaussian distributions gives us a simple analytical solution.

In [145]:
xobs = 5.
xb = 0


What is the posterior mean?

This is trivial from Gaussian stats and variational analysis, and you end up with:

$$
x=\frac{\frac{x_{obs}}{σ_{obs}^2} + \frac{x_b}{σ_b^2}}{\frac{1}{σ_{obs}^2} + \frac{1}{σ_b^2}}
$$

Which is just a reciprocal-uncertainty weighted average. 



In [146]:
mean,std = varsolution(xobs,xb)
print(f'mean {mean:.2f} sd {std:.3f}')

mean 2.50 sd 0.707


This makes sense, intuitively as well: the inputs have the same uncertainty, so the posterior will be half-way between prior and observation. The output uncertainty from a mean should be better than from any one sample (1/√N). This is the (variational) Data Assimilation result. This is what you have said you are solving. There is no judgement here on whether its good or bad. This is the solution according to the assumptions made.

# Whjat are you doing in what you claim is variational DA

Now take your approach to this problem:

	You randomly sample the prior and take the closest M matches
	You generate posterior statistics from those matches

You generate e.g. 1e5 samples over $B$. You think you need that many because otherwise we never sample at x=100, because from the prior its pretty unlikely. 

In [147]:
# your solution
xb_s = normal(xb,size=100000)
xobs_s = xobs

samples = closest(xb_s,xobs_s,M=5)
mean,std = np.mean(samples),np.std(samples)
print(f'mean {mean:.2f} sd {std:.3f}')

2336 samples within spec.
mean 4.84 sd 1.212


This is clearly crazy as a result. Its nowhere near the analytical solution we calculate above. If you run the cell again, you'll also see that the estimate of the mean and standard deviation is pretty unstable.

Hacking around
=============

You think maybe you haven't used enough samples. Or maybe $M$ is too low. Let's change $M$ as we have 2200-odd samples within spec, let's put it at 2000 here:

In [148]:
# your solution M=2000
xb_s = normal(xb,size=100000)
xobs_s = xobs

samples = closest(xb_s,xobs_s,M=2000)
print(samples)
mean,std = np.mean(samples),np.std(samples)
print(f'mean {mean:.2f} sd {std:.3f}')

2255 samples within spec.
[1.57554051 1.8184968  1.86999191 ... 8.32948919 8.47914554 9.20409533]
mean 5.01 sd 1.011


Hmmm... thats not right either. Maybe its sample number: let's put that high now, and increase M at the same time. Lets test stability whilst we do that:

In [149]:
for i in range(4):
    xb_s = normal(xb,size=10000000)
    samples = closest(xb_s,xobs_s,M=200000)
    mean,std = np.mean(samples),np.std(samples)
    print(f'mean {mean:.2f} sd {std:.3f}')

227581 samples within spec.
mean 5.00 sd 1.002
227999 samples within spec.
mean 5.00 sd 1.001
227732 samples within spec.
mean 5.00 sd 0.999
227839 samples within spec.
mean 5.00 sd 0.999


Maybe I shouldn't have put M up ...

In [150]:
for i in range(4):
    xb_s = normal(xb,size=10000000)
    samples = closes
    
    
    
    t(xb_s,xobs_s,M=5)
    mean,std = np.mean(samples),np.std(samples)
    print(f'mean {mean:.2f} sd {std:.3f}')

228005 samples within spec.
mean 5.83 sd 0.475
227528 samples within spec.
mean 4.85 sd 1.011
227769 samples within spec.
mean 5.24 sd 0.427
227762 samples within spec.
mean 4.86 sd 1.075


Nope. "OK, well, at least the one with high M is quite stable now, and gives a solution thats close to my observation, so it must be ok, right? Wasn't that what I really wanted in the first place, rather than the variational compromise I said I was looking for? The posterior matches my measurement of state, so this is perfect then."

**NO** That is just hacking around the mathematics, and making post-hoc decisions about what you wanted. It is not rigorous. It is not a solution to the problem that was laid out.

In fact, it is, in essence, **a solution based only on the observation**, and has no real sensitivity to the prior. If that's what you wanted, then you should have said so at the start. That might be appropriate in some cases. 

But its **not** data assimilation, is it? and the solution is not (much) constrained by the prior. If you had wanted to assume a weaker prior, thats what you should have done.

And also, it will only be appropriate when there is enough information in the observations to properly constrain the estimates of state. That is *not* the case in remote sensing, where we believe we have an ill-posed problem. An ill-posed problem is one **that is very sensitive to noise**, so the various efforts you have done in this, fitting to a single observation, make even less sense to me.

"But isn't that just what other people do in ensemble methods" you say?

**NO** Not if they are trying to solve the same problem we have laid out, unless they are just hacking around themselves, and shouldn't be publishing such things.

